In [1]:
# Only for debug
# !hadoop fs -rm -r hdfs://cluster-cc4766-3nodes-m/p2t1_receiver
# !hadoop fs -rm -r hdfs://cluster-cc4766-3nodes-m/p2t1_output

Deleted hdfs://cluster-cc4766-3nodes-m/p2t1_receiver
Deleted hdfs://cluster-cc4766-3nodes-m/p2t1_output


In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-46f9be11-18b0-4585-ab9c-012be26631b1;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 322ms :: artifacts dl 9ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [2]:
from pyspark.sql.functions import udf,collect_list,col,lit
from pyspark.sql.types import StructType, StructField, FloatType, StringType

In [3]:
!hadoop dfs -mkdir -p hdfs://cluster-cc4766-3nodes-m/p2t1_receiver_whole
!hadoop dfs -ls /



Found 7 items
-rw-r--r--   2 ecbm4040 hadoop   702821930 2022-04-26 21:22 /enwiki_small.xml
-rw-r--r--   2 ecbm4040 hadoop  1785209833 2022-04-26 21:23 /enwiki_test.xml
-rw-r--r--   2 ecbm4040 hadoop 30565654976 2022-04-26 21:36 /enwiki_whole.xml
drwxr-xr-x   - root     hadoop           0 2022-04-30 05:23 /p2t1_receiver_whole
drwxrwxrwt   - hdfs     hadoop           0 2022-04-29 07:25 /tmp
drwxrwxrwt   - hdfs     hadoop           0 2022-04-26 21:39 /user
drwx-wx-wx   - hive     hadoop           0 2022-04-26 21:18 /var


In [4]:
userSchema = StructType().add("lower_title", "string").add("rank", "double")

In [5]:
df = spark \
    .readStream \
    .option("sep", "\t") \
    .schema(userSchema) \
    .csv("hdfs://cluster-cc4766-3nodes-m/p2t1_receiver_whole/") 

In [6]:
df = df.select("lower_title", "rank").where("rank>0.5")

In [7]:
!hadoop dfs -mkdir -p hdfs://cluster-cc4766-3nodes-m/p2t1_output_whole
!hadoop dfs -ls /



Found 8 items
-rw-r--r--   2 ecbm4040 hadoop   702821930 2022-04-26 21:22 /enwiki_small.xml
-rw-r--r--   2 ecbm4040 hadoop  1785209833 2022-04-26 21:23 /enwiki_test.xml
-rw-r--r--   2 ecbm4040 hadoop 30565654976 2022-04-26 21:36 /enwiki_whole.xml
drwxr-xr-x   - root     hadoop           0 2022-04-30 05:23 /p2t1_output_whole
drwxr-xr-x   - root     hadoop           0 2022-04-30 05:23 /p2t1_receiver_whole
drwxrwxrwt   - hdfs     hadoop           0 2022-04-29 07:25 /tmp
drwxrwxrwt   - hdfs     hadoop           0 2022-04-26 21:39 /user
drwx-wx-wx   - hive     hadoop           0 2022-04-26 21:18 /var


In [8]:
query = df.writeStream \
    .format("csv") \
    .option("delimiter", "\t") \
    .option("checkpointLocation", "cluster-cc4766-3nodes-m/p2t1_output_whole/") \
    .option("path", "cluster-cc4766-3nodes-m/p2t1_output_whole/") \
    .start()

22/04/30 05:23:27 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
def pageRank(inputPath, outputPath):


    tmp = spark.read.format('csv').options(delimiter='\t').load(inputPath)

    from pyspark.sql.functions import udf,collect_list,col,lit

    tmp = tmp.toDF('lower_title', 'link')
    tmp = tmp.dropna().sort('lower_title', 'link')

    def compute_contribs(urls, rank):

        num_urls = len(urls)
        for url in urls:
            yield (url, rank / num_urls)

    links = tmp.groupBy('lower_title').agg(collect_list("link")).toDF('lower_title',"links").sort('lower_title')
    ranks = links.select(col("lower_title")).withColumn("rank",lit(1))

    from operator import add

    iter_num = 10
    for i in range(iter_num):
        contribs=links.join(ranks,on="lower_title").rdd.flatMap(lambda url_urls_rank: compute_contribs(url_urls_rank[1], url_urls_rank[2]))
        ranks = contribs.reduceByKey(add).mapValues(lambda rank : rank * 0.85 + 0.15)
        ranks = spark.createDataFrame(data = ranks, schema = ["lower_title", "rank"])
        print("The {}-th iteration completed.".format(i+1))

    df_output = ranks.sort("lower_title", "rank")

    df_output.coalesce(1).write.option("delimiter", "\t").csv(outputPath, mode='overwrite')
                                                                
    print("Output finished.")


In [9]:
pageRank("gs://dataproc-staging-us-central1-543716498799-fp7nsgsr/task2whole", "hdfs://cluster-cc4766-3nodes-m/p2t1_receiver_whole/")

The 1-th iteration completed.


The 2-th iteration completed.


The 3-th iteration completed.


The 4-th iteration completed.


The 5-th iteration completed.


...

The 7-th iteration completed.


The 8-th iteration completed.


The 9-th iteration completed.


The 10-th iteration completed.


Output finished.


In [12]:
query.stop()

In [19]:
p2t1_df = spark.read.format('csv').schema(userSchema).options(delimiter='\t').load("hdfs://cluster-cc4766-3nodes-m/p2t1_output_whole/")
print("The number of articles whose rank is greater than 0.5 is {}".format(p2t1_df.count()))

The number of articles whose rank is greater than 0.5 is 1163257


In [21]:
spark.stop()